In [90]:
'''
Select an individual feature by attribute value from one feature class, then select features from another feature class by distance from the original feature. 
Originally used for selecting census block groups within a specified distance of a point feature. 
'''
def select_feature_by_attribute(focus_features, focus_field, focus_value): 
    
    # Construct SQL 
    sql = f"{focus_field} = '{focus_value}'"

    # Apply attribute selection 
    focus_point, count          = arcpy.management.SelectLayerByAttribute(focus_features, 'NEW_SELECTION', sql)
    if count == '0': 
        print('No focus features selected')
    elif count != 1: 
        print(f'Feature selected: {focus_value}')
        pass
    else: 
        print(f'Multiple features selected: {count}')
        
    return focus_point

In [91]:
'''
Create a dictionary of distance windows to export by 
'''
def create_selection_distance_dictionary(starting_hi, ending_hi, interval): 
    
    selection_distance_dictionary = {} 

    for i in range(starting_hi, ending_hi, interval): 
        selection_distance_dictionary[i] = i + interval
        
    return selection_distance_dictionary

In [92]:
'''
Export geographic features using the selection distance dictionary 
'''
def export_from_selection_distance_dictionary(selection_distance_dictionary, polygon_features, focus_point, focus_value, polygon_output_folder): 
    
    # Initialize export paths 
    print(f'Output: {focus_value}')
    
    # Iterate through dictionary 
    for low_distance, high_distance in selection_distance_dictionary.items(): 

        # High selection distance 
        selection_distance = f"{high_distance} Miles"
        print("---------")
        print(selection_distance)
        print("---------")

        # Make initial selection 
        high_selection, merge, high_count = arcpy.management.SelectLayerByLocation(polygon_features, 'WITHIN_A_DISTANCE', focus_point, selection_distance)

        if low_distance != 0: 
            # Low selection distance 
            selection_distance = f"{low_distance} Miles"

            # Remove selection 
            low_selection, merge, low_count  = arcpy.management.SelectLayerByLocation(high_selection, 'WITHIN_A_DISTANCE', focus_point, selection_distance, selection_type = 'REMOVE_FROM_SELECTION')
            
            # Define export path - create folder if needed 
            export_folder = os.path.join(polygon_output_folder, focus_value)
            export_path   = os.path.join(export_folder, f'{focus_value}_{high_distance}.shp')

            if arcpy.Exists(export_folder) == False: 
                os.mkdir(export_folder)
                
            # Export features 
            arcpy.conversion.ExportFeatures(low_selection, export_path)

        else: 
            # Define export path - create folder if needed 
            export_folder = os.path.join(polygon_output_folder, focus_value)
            export_path   = os.path.join(export_folder, f'{focus_value}_{high_distance}.shp')

            if arcpy.Exists(export_folder) == False: 
                os.mkdir(export_folder)
                
            # Export features 
            arcpy.conversion.ExportFeatures(high_selection, export_path)
                
        print(export_path)
        
    return export_path

In [93]:
#######################################################################################################################################
#######################################################################################################################################
#######################################################################################################################################

In [94]:
'''
ACS 1 - MASTER
'''
def create_geography(focus_features, focus_field, focus_value, starting_hi, ending_hi, interval, polygon_features, polygon_output_folder): 
    
    # Select focus feature 
    focus_point                   = select_feature_by_attribute(focus_features, focus_field, focus_value)
    
    # Create selection distance dictionary 
    selection_distance_dictionary = create_selection_distance_dictionary(starting_hi, ending_hi, interval)
    
    # Export from selection distance dictionary 
    export_paths                  = export_from_selection_distance_dictionary(selection_distance_dictionary, polygon_features, focus_point, focus_value, polygon_output_folder)
    
    return export_paths 